# Bernoulli Matrix Factorization

En líneas generales, los algoritmos de factorización matricial vistos hasta el momento funcionan de forma correcta en la mayoría de los casos. Sin embargo, estos algoritmos están fundamentados matemáticamente en un supuesto erróneo: las votaciones en un sistema de recomendación son discretas y no continuas. Dicho de otro modo, PMF asume que la distribución que subyace a los votos es gaussiana y, por tanto, se permite cualquier valor de votación (por ejemplo, 3.4) a pesar de que las votaciones habitualmente están restringidas (por ejemplo, 1, 2, 3, 4 ó 5).

Para solventar este problema, Bernoulli Matrix Factorization (BeMF) plantea utilizar una distribución de probabilidad discreta, la distribución de Bernoulli, para representar los votos. La distribución de Bernoulli requiere que los datos de entrada sean binarios (0 ó 1, verdadero o falso), por lo que, para adaptarla al problema del filtrado colaborativo, se requiere pre-procesar la matriz de votos.

La siguiente figura resume el funcionamiento de BeMF. El *dataset* de entrada es dividido en tantas matrices (dispersas) como posibles votaciones (*scores*) haya (en el caso de la imagen 5 matrices). Cada una de esas matrices contiene una representación binaria del voto, teniendo el valor 0 si el usuario no voto el ítem con la puntación del *score* y 1 en caso contrario. Posteriormente, se aplica un proceso de factorización matricial a cada una de estas matrices asumiendo que los votos siguen una distribución de Bernoulli. Finalmente, los resultados son combinados para obtener la predicción del algoritmo, que será la que obtenga la puntuación (estimación) más alta.

<img src=https://i.ibb.co/58f4n0P/Be-MF-architecture.png style='background-color: white'>

Una gran ventaja de este algoritmo, más allá de su robustez matemática, es que no proporciona únicamente predicciones, si no que indica, además, la fiabilidad de estas predicciones. Esto permite modular la salida del modelo para proporcionar no solo predicciones altas si no también fiables. ¿Es mejor una predicción de 5 con una fiabilidad de 0,5 o una predicción de 4 con una fiabilidad de 0,9?

Entendido el algoritmo, debemos determinar cómo calculamos las factorizaciones de Bernoulli para cada *score*. Para ello fijamos $s \in \mathcal{S}$ como posibles votaciones. Para aliviar la notación llamaremos $R = R^s$ a la matriz de votaciones del *score* $s$.

Definimos una función logística $\psi: \mathbb{R} \to [0,1]$ monótona creciente que cumpla $\psi(x) \to 0$ cuando    $x \to -\infty$ y $\psi(x) \to 1$ cuando $x \to \infty$.

Dado el usuario $u$ y el ítem $i$ se asume que sus votaciones $R_{u,i}$ siguen una distribución de Bernoulli con una probabilidad de éxito $\psi(U_u \cdot V_i)$, siendo usuario $U_u$ y $V_i$ la representación latente del usuario $u$ y el ítem $i$ en vectores de dimensión $k>0$.

La función de masa de esta variable aleatoria $p(R_{u,i} | U_u, V_i)$ viene dada por:

$$
        p(R_{u,i} | U_u, V_i) = \left\{\begin{matrix}\psi(U_uV_i) & \textrm{if } R_{u,i} = 1, \\ 1-\psi(U_uV_i) & \textrm{if } R_{u,i} = 0.\end{matrix}\right.
$$

De esta fórmula podemos obtener la función de verosimilitud:

$$
L(R | U, V) = \prod_{R_{u,i} \neq \bullet} p(R_{u,i} | U_u, V_i) = \left(\prod_{R_{u,i} = 1} \psi(U_uV_i)\right)\left(\prod_{R_{u,i} = 0} 1-\psi(U_uV_i)\right).
$$

Y el la log-verosimilitud $l(R | U, V) = \log L(R | U, V)$:

$$
l(R | U, V) =    \sum_{R_{u,i} = 1} \log(\psi(U_uV_i)) + \sum_{R_{u,i} = 0} \log(1-\psi(U_uV_i)).
$$

Dada esta función, y asumiendo las distribuciones a priori $U$ y $V$ como gaussianas de media 0 y desviaciones típicas $\sigma_U, \sigma_V > 0$, sus funciones de densidad son:

$$
        p(U_u) = \frac{1}{\sigma_U \sqrt{2\pi}} \exp\left(-\frac{||U_u||^2}{2\sigma_U^2}\right), \quad p(V_i) = \frac{1}{\sigma_V \sqrt{2\pi}} \exp\left(-\frac{||V_i||^2}{2\sigma_V^2}\right).
$$

Y las de verosimilitud:

$$
        L(U) = \prod_{u=1}^N p(U_u) = \frac{1}{\sigma_U^N (2\pi)^{N/2}} \prod_{u=1}^N \exp\left(-\frac{||U_u||^2}{2\sigma^2_U}\right) = \frac{1}{\sigma_U^N (2\pi)^{N/2}}    \exp\left(-\frac{\sum_{u=1}^N ||U_u||^2}{2\sigma^2_U}\right),
$$
$$
        L(V) = \prod_{i=1}^M p(V_i) = \frac{1}{\sigma^M_V (2\pi)^{M/2}} \prod_{i=1}^M \exp\left(-\frac{||V_i||^2}{2\sigma_V^2}\right) = \frac{1}{\sigma_V^M (2\pi)^{M/2}}    \exp\left(-\frac{\sum_{i=1}^M ||V_i||^2}{2\sigma_V^2}\right).
$$

Y las de log-verosimilitud:

$$
        l(U) = -\frac{1}{2\sigma_U^2}\sum_{u=1}^N ||U_u||^2 + C_U, \quad
        l(V) = -\frac{1}{2\sigma_V^2}\sum_{i=1}^M ||V_i||^2 + C_V.
$$

para las constantes $C_U = -N\log(\sigma_U \sqrt{2\pi})$ y $C_V = -M\log(\sigma_V \sqrt{2\pi})$.

Finalmente, la función de verosimilitud a posteriori, $L(R)$, es

$$
        L(R) = L(R | U, V) L(U) L(V).
$$

Y la log-verosimilitud es

\begin{align*}
l(R) &= l(R |U,V) + l(U) + l(V) \\
&= \sum_{R_{u,i} = 1} \log(\psi(U_uV_i)) + \sum_{R_{u,i} = 0} \log(1-\psi(U_uV_i)) \\ &\hspace{0.5cm}-\frac{1}{2\sigma_U^2}\sum_{u=1}^N ||U_u||^2 -\frac{1}{2\sigma_V^2}\sum_{i=1}^M ||V_i||^2 + C,
\end{align*}

donde $C = C_U + C_V$ es una constante

El **estimador de máxima verosimilitud** se obtiene maximizando esta log-verosimilitud a posteriori. Para este propósito, la constante $C$ es irrelevante y puede ser omitida. Fijando $\eta_U = \frac{1}{\sigma_U^2}$ y $\eta_V = \frac{1}{\sigma_V^2}$, el problema de maximización puede ser convertido a minimizar la función de coste

$$
F(U, V) = -\sum_{R_{u,i} = 1} \log(\psi(U_uV_i)) - \sum_{R_{u,i} = 0} \log(1-\psi(U_uV_i)) + \frac{\eta_U}{2}\sum_{u=1}^N ||U_u||^2 + \frac{\eta_V}{2}\sum_{i=1}^M ||V_i||^2.
$$

Para optimizar esta función de coste usaremos **descenso de gradiente**. Fijado el usuario $u_0$ y el ítem $i_0$, denotando sus factores latente $U_{u_0} = (U_{u_0, 1}, \ldots, U_{u_0, k})$ y $V_{i_0} = (V_{i_0, 1}, \ldots, V_{i_0, k})$. Las derivadas parciales de $F$ con respecto a $U_{u_0, a}$ y $V_{i_0, b}$ vienen dadas por

$$
        \frac{\partial F}{\partial U_{u_0,a}} = -\sum_{\left\{i \,|\, R_{u_0,i} = 1\right\}} \frac{\psi'(U_{u_0}V_i)}{\psi(U_{u_0}V_i)}V_{i,a} + \sum_{\left\{i \,|\, R_{u_0,i} = 0\right\}} \frac{\psi'(U_{u_0}V_i)}{1-\psi(U_{u_0}V_i)}V_{i,a} + \eta_U U_{u_0,a},
$$
$$
        \frac{\partial F}{\partial V_{i_0,b}} = -\sum_{\left\{u \,|\, R_{u,i_0} = 1\right\}} \frac{\psi'(U_{u}V_{i_0})}{\psi(U_{u}V_{i_0})}U_{u,b} + \sum_{\left\{u \,|\, R_{u,i_0} = 0\right\}} \frac{\psi'(U_{u}V_{i_0})}{1-\psi(U_{u}V_{i_0})}U_{u,b} + \eta_V V_{i_0,b}.
$$

Por simplificar, tomamos $\eta_U = \eta_V = \eta$. Los pasos del descenso de gradiente de paso $\gamma > 0$ para la iteración $T+1$ son

$$
        U_u^{T+1} = U_u^{T} + \gamma \left(\sum_{\left\{i \,|\, R_{u,i} = 1\right\}} \frac{\psi'(U_{u}V_i)}{\psi(U_{u}V_i)}V_{i} - \sum_{\left\{i \,|\, R_{u,i} = 0\right\}} \frac{\psi'(U_{u}V_i)}{1-\psi(U_{u}V_i)}V_{i} - \eta U_{u}\right),
$$
$$
        V_i^{T+1} = V_i^{T} + \gamma \left(\sum_{\left\{u \,|\, R_{u,i} = 1\right\}} \frac{\psi'(U_{u}V_{i})}{\psi(U_{u}V_{i})}U_{u} - \sum_{\left\{u \,|\, R_{u,i} = 0\right\}} \frac{\psi'(U_{u}V_{i})}{1-\psi(U_{u}V_{i})}U_{u} - \eta V_{i}\right).
$$

Si usamos como función logística $\psi(x) = logit(x) = \frac{1}{1+e^{-x}}$, donde $logit'(x) = logit(x) (1-logit(x))$, entonces las ecuaciones de actualización se simplifican como

$$
        U_u^{T+1} = U_u^{T} + \gamma \left(\sum_{\left\{i \,|\, R_{u,i} = 1\right\}} (1-\mathrm{logit}(U_{u}V_i))V_{i} - \sum_{\left\{i \,|\, R_{u,i} = 0\right\}} \mathrm{logit}(U_{u}V_i)V_{i} - \eta U_{u}\right),
$$
$$
        V_i^{T+1} = V_i^{T} + \gamma \left(\sum_{\left\{u \,|\, R_{u,i} = 1\right\}} (1-\mathrm{logit}(U_{u}V_i))U_{u} - \sum_{\left\{u \,|\, R_{u,i} = 0\right\}} \mathrm{logit}(U_{u}V_i)U_{u} - \eta V_{i}\right).
$$

## Carga del dataset

Para ilustar mejor el funcionamiento el algoritmo BeMF, vamos a desarrollar una implementación del mismo.

Para ello usaremos el dataset de [MovieLens 100K](https://grouplens.org/datasets/movielens/) que contiene 100.000 votos de 943 usuarios sobre 1682 películas. Este dataset ha sido dividido en votaciones de entrenamiento (80%) y votaciones de test (20%). Además, los códigos de usuarios e items han sido modificados para que comience en 0 y terminen en el número de (usuarios / items) - 1.

Inicialmente definimos algunas constantes que nos serán necesarias durante la codificación del algoritmo:

In [1]:
import urllib.request
import random
import math
import numpy as np

In [150]:
NUM_USERS = 943
NUM_ITEMS = 1682

MIN_RATING = 1
MAX_RATING = 5

SCORES = np.array([1, 2, 3, 4, 5])

Y cargamos la matriz con las votaciones de entrenamiento:

In [3]:
ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

training_file = urllib.request.urlopen("https://drive.upm.es/s/tDdluElfGInyUnU/download")
for line in training_file:
    [u, i, rating] = line.decode("utf-8").split("::")
    ratings[int(u)][int(i)] = int(rating)

In [4]:
ratings_np = np.array(ratings)
ratings_np[ratings_np == None] = np.nan
ratings_np = ratings_np.astype(np.float64)

Del mismo modo, cargamos la matriz de votaciones de test:

In [5]:
test_ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

test_file = urllib.request.urlopen("https://drive.upm.es/s/Jn75Vg6okOPsgZu/download")
for line in test_file:
    [u, i, rating] = line.decode("utf-8").split("::")
    test_ratings[int(u)][int(i)] = int(rating)

In [6]:
test_ratings_np = np.array(test_ratings)
test_ratings_np[test_ratings_np == None] = np.nan
test_ratings_np = test_ratings_np.astype(np.float64)

## Inicialización del modelo

Definimos los parámetros necesarios para implementar la factorización matricial mediante BeMF.

In [7]:
NUM_FACTORS = 7
LEARNING_RATE = 0.001
REGULARIZATION = 0.1

Inicializamos las matrices de factores con valores uniformes aleatorios en el intervalo \[0, 1].

In [8]:
U = [[[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] for _ in range(len(SCORES))]
V = [[[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] for _ in range(len(SCORES))]

In [9]:
U_np = np.array(U) # np.random.rand(len(SCORES), NUM_USERS, NUM_FACTORS)
V_np = np.array(V) # np.random.rand(len(SCORES), NUM_ITEMS, NUM_FACTORS)

## Cálculo de las predicciones

A diferencia de otros modelos de factorización matricial, el cálculo de las predicciones implica encontrar la puntuación $s$ en la que se maximiza la probabilidad del voto:

$$
\hat{r}_{u,i} = arg \max_{s} \psi(U_u^s \cdot V_i^s)
$$

La siguiente función realiza esta operación:


In [10]:
def logit (x):
    return 1 / (1 + math.exp(-x))

In [39]:
def logit_np(x):
    return 1 / (1 + np.exp(-x))

In [11]:
def compute_prediction(u, i):
    prediction = None
    prob = 0
    for s in range(len(SCORES)):
        dot = np.dot(U[s][u], V[s][i])
        if logit(dot) > prob:
            prob = logit(dot)
            prediction = SCORES[s]
    return prediction, prob

## Aprendizaje de los factores latentes

El proceso de entrenamiento implicar aplicar las operaciones de actualización de las matrices de factores hasta que el algoritmo converja. En general, esta convergencia suele prefijarse como el número de iteraciones que realizamos sobre las operaciones de actualización:

In [67]:
NUM_ITERATIONS = 10

El siguiente código ejemplifica el proceso de entrenamiento del algoritmo:

In [ ]:
5:18

In [ ]:
for it in range(NUM_ITERATIONS):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

    for s in range(len(SCORES)):

        # Update users
        for u in range(NUM_USERS):
            delta = np.zeros(NUM_FACTORS)
            for i in range(NUM_ITEMS):
                dot = np.dot(U[s][u], V[s][i])
                for f in range(NUM_FACTORS):
                    if ratings[u][i] == SCORES[s]:
                        delta[f] += (1 - logit(dot)) * V[s][i][f]
                    else:
                        delta[f] -= logit(dot) * V[s][i][f]

            delta -= REGULARIZATION * np.asarray(U[s][u])
            delta *= LEARNING_RATE
            U[s][u] += delta


        for i in range(NUM_ITEMS):
            delta = np.zeros(NUM_FACTORS)
            for u in range(NUM_USERS):
                dot = np.dot(U[s][u], V[s][i])
                for f in range(NUM_FACTORS):
                    if ratings[u][i] == SCORES[s]:
                        delta[f] += (1 - logit(dot)) * U[s][u][f]
                    else:
                        delta[f] -= logit(dot) * U[s][u][f]
            
            delta -= REGULARIZATION * np.asarray(V[s][i])
            delta *= LEARNING_RATE
            V[s][i] += delta

In [176]:
# Hay un problema con la vectorización
# Esto es que los factores sufren "RAW" y "WAR"
# Por tanto no se puede vectorizar totalmente sin perder el funcionamiento 
# intencionado que es que vaya dando pasos por cada usuario y cada item en vez 
# de dar un paso global para usuarios y otro para items

for it in range(NUM_ITERATIONS):
    print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

    for s in range(len(SCORES)):

        # Update users
        for u in range(NUM_USERS):
            dots = V_np[s] @ U_np[s][u].T
            dots_logit = logit_np(dots)
            positive = ratings_np[u] == SCORES[s]
            delta = ((positive * (1 - dots_logit))[:, np.newaxis] * V_np[s] - ((1 - positive) * dots_logit)[:, np.newaxis] * V_np[s]).sum(axis=0)
            delta -= REGULARIZATION * U_np[s][u]
            delta *= LEARNING_RATE
            U[s][u] += delta

        # Update items
        for i in range(NUM_ITEMS):
            dots = U_np[s] @ V_np[s][i].T
            dots_logit = logit_np(dots)
            positive = ratings_np[:,i] == SCORES[s]
            delta = ((positive * (1 - dots_logit))[:, np.newaxis] * U_np[s] - ((1 - positive) * dots_logit)[:, np.newaxis] * U_np[s]).sum(axis=0)
            delta -= REGULARIZATION * V_np[s][i]
            delta *= LEARNING_RATE
            V[s][i] += delta

Iteración 1 de 10
Iteración 2 de 10
Iteración 3 de 10
Iteración 4 de 10
Iteración 5 de 10
Iteración 6 de 10
Iteración 7 de 10
Iteración 8 de 10
Iteración 9 de 10
Iteración 10 de 10


## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|     	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|     	|    2,9 	|        	|    4,7 	|    5,0 	|        	|    1,2 	|        	|     	|    3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.

In [87]:
N = 5

In [88]:
def get_recommendations (predictions):
    recommendations = [None for _ in range(N)]

    for n in range(N):

        max_value = 0
        item = None

        for i, value in enumerate(predictions):
            if i not in recommendations and value != None and value > max_value:
                max_value = value
                item = i
        recommendations[n] = item
    return recommendations

##Ejercicio: Cálculo de Métricas

Calcular el error medio absoluto (MAE) y la raiz del error medio cuadrático (RMSE) de las predicciones realizadas por el algoritmo BeMF, así como la precisión, recall, F1 y nDCG de las recomendaciones.

Para ello, lo primero que debemos hacer es calcular las predicciones para todos los items que haya recibido una votación de test:

In [154]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

# Rellenamos la matriz de predicciones
for u in range(NUM_USERS):
    for i in range(NUM_ITEMS):
        if test_ratings[u][i] != None:
            pred, prob = compute_prediction(u, i)
            predictions[u][i] = pred

In [153]:
def compute_prediction(u, i):
    prediction = None
    prob = 0
    for s in range(len(SCORES)):
        dot = np.dot(U[s][u], V[s][i])
        if logit(dot) > prob:
            prob = logit(dot)
            prediction = SCORES[s]
    return prediction, prob

In [177]:
# Compute predictions
logits = np.einsum('suf,sif->uis', U_np, V_np)
probs = logit_np(logits)
probs = probs / probs.sum(axis=-1)[..., np.newaxis] # Not needed, but interesting data
predictions = SCORES[np.argmax(probs, axis=-1)].astype(object)
predictions[np.isnan(test_ratings_np)] = None
predictions

array([[None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None]], dtype=object)

Y, a continuación, calculamos las métricas:



In [158]:
def get_user_mae (u, predictions):
  mae = 0
  count = 0

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mae += abs(test_ratings[u][i] - predictions[u][i])
      count += 1

  if count > 0:
    return mae / count
  else:
    return None

In [159]:
def get_mae (predictions):
  mae = 0
  count = 0

  for u in range(NUM_USERS):
    user_mae = get_user_mae(u, predictions)

    if user_mae != None:
      mae += user_mae
      count += 1

  if count > 0:
    return mae / count
  else:
    return None

In [160]:
import math

def get_user_rmse (u, predictions):
  mse = 0
  count = 0

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      mse += (test_ratings[u][i] - predictions[u][i]) * (test_ratings[u][i] - predictions[u][i])
      count += 1

  if count > 0:
    return math.sqrt(mse / count)
  else:
    return None

In [161]:
def get_rmse (predictions):
  rmse = 0
  count = 0

  for u in range(NUM_USERS):
    user_rmse = get_user_rmse(u, predictions)

    if user_rmse != None:
      rmse += user_rmse
      count += 1


  if count > 0:
    return rmse / count
  else:
    return None

In [162]:
theta = 4

In [163]:
def get_user_precision (u, predictions):
  precision = 0
  count = 0
  recommendations = get_recommendations(predictions[u])

  for i in recommendations:
    if i != None and test_ratings[u][i] != None:
      precision += 1 if test_ratings[u][i] >= theta else 0
      count += 1

  if count > 0:
    return precision / count
  else:
    return None

In [164]:
def get_precision (predictions):
  precision = 0
  count = 0

  for u in range(NUM_USERS):
    user_precision = get_user_precision(u, predictions)

    if user_precision != None:
      precision += user_precision
      count += 1


  if count > 0:
    return precision / count
  else:
    return None

In [165]:
def get_user_recall (u, predictions):
  recall = 0
  count = 0
  recommendations = get_recommendations(predictions[u])

  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None and predictions[u][i] != None:
      if test_ratings[u][i] >= theta:
        recall += 1 if i in recommendations else 0
        count += 1

  if count > 0:
    return recall / count
  else:
    return None

In [166]:
def get_recall (predictions):
  recall = 0
  count = 0

  for u in range(NUM_USERS):
    user_recall = get_user_recall(u, predictions)

    if user_recall != None:
      recall += user_recall
      count += 1


  if count > 0:
    return recall / count
  else:
    return None

In [167]:
def get_user_f1 (u, predictions):
  precision = get_user_precision(u, predictions)
  recall = get_user_recall(u, predictions)

  if precision == None or recall == None:
    return None
  elif precision == 0 and recall == 0:
    return 0
  else:
    return 2 * precision * recall / (precision + recall)

In [168]:
def get_f1 (predictions):
  f1 = 0
  count = 0

  for u in range(NUM_USERS):
    user_f1 = get_user_f1(u, predictions)

    if user_f1 != None:
      f1 += user_f1
      count += 1


  if count > 0:
    return f1 / count
  else:
    return None

In [169]:
def get_ordered_test_items(u):
  num_items = sum(x is not None for x in test_ratings[u])
  items = [None for _ in range(num_items)]

  for n in range(num_items):

    max_value = 0
    item = None

    for i,value in enumerate(test_ratings[u]):
      if i not in items and value != None and value > max_value:
        max_value = value
        item = i

    items[n] = item

  return items

In [170]:
def get_user_idcg (u):
  items = get_ordered_test_items(u)
  idcg = 0

  for pos, i in enumerate(items):
    idcg += (2 ** test_ratings[u][i] - 1) / math.log(pos+2, 2)

  return idcg

In [171]:
def get_user_dcg (u, recommendations):
  dcg = 0

  for pos, i in enumerate(recommendations):
    if i != None and test_ratings[u][i] != None:
      dcg += (2 ** test_ratings[u][i] - 1) / math.log(pos+2, 2)

  return dcg

In [172]:
def get_user_ndcg (u, predictions):
  recommendations = get_recommendations(predictions[u])
  dcg = get_user_dcg(u, recommendations)
  idcg = get_user_idcg(u)
  if idcg == 0:
    return 0
  else:
    return dcg / idcg

In [173]:
def get_ndcg (predictions):
  ndcg = 0
  count = 0

  for u in range(NUM_USERS):
    user_ndcg = get_user_ndcg(u, predictions)

    if user_ndcg != None:
      ndcg += user_ndcg
      count += 1


  if count > 0:
    return ndcg / count
  else:
    return None

In [178]:
mae = get_mae(predictions)
rmse = get_rmse(predictions)
precision = get_precision(predictions)
recall = get_recall(predictions)
f1 = get_f1(predictions)
ndcg = get_ndcg(predictions)
print("MAE = " + str(mae))
print("RMSE = " + str(rmse))
print("Precision = " + str(precision))
print("Recall = " + str(recall))
print("F1 = " + str(f1))
print("nDCG = " + str(ndcg))

MAE = 1.5451816695559673
RMSE = 1.869397693862923
Precision = 0.5878632478632477
Recall = 0.47579313695927167
F1 = 0.45816952268787436
nDCG = 0.10226429215421418


## Referencias

Ortega, F., Lara-Cabrera, R., González-Prieto, Á., & Bobadilla, J. (2021). **Providing reliability in recommender systems through Bernoulli Matrix Factorization**. Information Sciences, 553, 110-128.
